**Nota**: Leer antes el archivo "memoria_TFM.pdf" 

# TFM CÁLIDAD DEL USUARIO

## Este notebook consta de las siguientes partes:

- Importación de librerías
- Preparación del data frame
- Análisis y estudio del proceso


In [1]:
import pandas as pd
import numpy as np

### Preparación del data frame

**Nota1**: La mayoría de los archivos que usaremos han sido previeamente modificados por motivos de privacidad, por lo que al importarlos usaremos *index_col=0* ya que ya tenían indice. Ninguna de estas modificaciones afecta al resultado del proyecto.

**Nota2**: Todos los archivos que importaremos son .csv y estaran en la misma carpeta de este notebook.

Importamos la primera consulta que hemos realizado (usuarios_proceso_y_so.csv) en google analytics. Este archivo contiene a los usuarios que han accedido al proceso y el sistema operativo desde el que han accedido (Mac OS o Windows OS). También tenemos otra columna de usuarios pero la eliminaremos ya que no nos es útil.

In [25]:
# Importamos usando pandas
usu_so = pd.read_csv('./usuarios_proceso_y_so.csv', index_col=0)
# Veamos que pinta tiene el archivo
print usu_so.head()
print ""
print 'Tenemos %i filas y %i columnas' %(usu_so.shape[0], usu_so.shape[1])

  identificador Sistema operativo  Usuarios
0       245701u        Windows OS         1
1         1120u        Windows OS         1
2       267373u            Mac OS         1
3       149066u        Windows OS         1
4        90480u        Windows OS         1

Tenemos 28960 filas y 3 columnas


In [26]:
# Modificamos el nombre de las columnas por motivos de uniformidad.
# El formato siempre sera: minúsculas, sin tildes y con barra baja como espacio.
usu_so.columns = ['identificador', 'so', 'usuarios']
# Eliminamos la columna usuarios ya que no nos aporta nada.
usu_so.drop('usuarios', 1, inplace = True)

Para empezar a preparar nuestro data frame debemos quedarnos con los identificadores únicos de usu_so. Obviamente un usuario ha podido acceder al proceso desde un solo sistema operativo o desde los dos, por tanto, tenemos que eliminar identificadores duplicados.

In [34]:
# Esto podemos hacerlo con pandas o con un bucle for con numpy.
# Bajo mi punto de vista la forma más fácil es con pandas.

# Con pandas
# Copiamos usu_so en un nuevo dataframe.
df = usu_so.copy()
# Ordenamos por la columna identificador y eliminamos los duplicados
df.sort_values('identificador', inplace=True)
df.drop_duplicates('identificador', inplace=True)
# Eliminamos la columna del sistema operativo para quedarnos solo con el identificador
df.drop('so', 1, inplace=True)

print 'Había %i usuarios duplicados' %(len(usu_so)-len(df))
print 'Han entrado %i usuarios en el periodo seleccionado en la consulta' %(len(df))

Había 125 usuarios duplicados
Han entrado 28835 usuarios en el periodo seleccionado en la consulta


In [ ]:
# Faltaría hacer la celda anterior con un bucle for en numpy

Ya tenemos la primera columna de nuestro data frame. Iremos añadiendo nuevas variables a este.

Como primera variable y aprovechando que tenemos importado el archivo de los sistemas operativos, crearemos una nueva variable que pueda tener 3 valores diferentes:
- 0 si el sistema operativo solo es Mac OS.
- 1 si el sistema operativo solo es Windows OS.
- 2 si el sistema operativo han sido ambos.

In [53]:
# Primero ordenamos el data frame por identificador de usuario
usu_so.sort_values('identificador', inplace=True)

sist_ope = [] # Creamos una lista vacía donde iremos añadiendo el resultado 0,1 o 2.
usuarios = [] # Creamos otra lista vacía donde iremos añadiendo los usuarios que ya tenemos.

# La idea es hacer un bucle que recorra una lista de tuplas e ir añadiendo los identificadores
# a la lista creada 'usuarios' y conforme añadamos usuarios, si el identificador ya está en la lista
# sabremos que ese usuario ha accedido desde ambos sistemas operativos y por tanto sera un 2 en 'sist_ope'
for usuario_so in usu_so.values.tolist():
    
    if usuario_so[0] in usuarios: # Con usuario_so[0] accedemos a el primer valor de la tupla
        
        # Si el usuario ya estaba cambiamos el ultimo valor añadido por un 2
        # El último valor añadido era 0 o 1 de ese mismo usuario.
        sist_ope[len(sist_ope)-1] = 2
        
    else:
        usuarios.append(usuario_so[0]) # Si no estaba en la lista de usuarios, lo añadimos
        if usuario_so[1] == 'Mac OS':  # Si es Mac OS añadimos un 0 a sist_ope
            sist_ope.append(0)
        else:
            sist_ope.append(1)         # Si no, será Windows OS y añadimos un 1

In [54]:
# Pendiente hacer una función

In [64]:
# Si hemos hecho bien el bucle anterior el número de valores 2 en sist_ope
# debería ser igual a 125 (usuarios duplicados)
print len ([l for l in sist_ope if l==2])

# len(sist_ope)  y len(usuarios) deberían ser igual que len(df)
print len(sist_ope)
print len(usuarios)
print len(df)

125
28835
28835


In [66]:
# Vamos a convertir 'sist_ope' y 'usuarios' en un dataframe para hacer un merfe por el identificador
temp = pd.DataFrame({'identificador':usuarios, 'ssoo':sist_ope})

# Unimos los dataframes y ya tenemos nuestra primera variable.
df = pd.merge(df, temp, on='identificador', how='left')

df.head()

,identificador,ssoo
0,100016u,0
1,100018u,0
2,100026u,1
3,100027u,1
4,100036u,1


Importamos la segunda consulta que hemos hecho en google analytics. En este archivo encontramos lás páginas del proceso que ha visitado cada usuario, el tiempo medio que ha estado en cada una de ellas, el número de veces que ha visto cada página y el número de visualizaciones únicas de cada página.

In [69]:
datos_proceso = pd.read_csv('./datos_proceso.csv', index_col=0)
datos_proceso.head()

,identificador,página,Promedio de tiempo en la página,Número de visitas a páginas,Número de visitas a páginas únicas
0,195567u,page1,1627.0,1,1
1,121943u,page1,1623.0,1,1
2,110307u,page2,1557.0,1,1
3,238327u,page1,1548.0,1,1
4,236613u,page2,1498.0,1,1


Vamos a tratar este archivo para añadir varias variables a nuestro dataframe. Empezaremos cálculando la profundidad del usuario, es decir, lo lejos que ha llegado el usuario en el proceso. El valor irá de 1 a 4 en función de si se quedo en la página 1,2,3 o 4.

In [75]:
# Puesto que un usuario aparece una vez para cada página que ha visto, para calcular la profundidad
# solo necesitamos la columna 'identificador' y contar cuantas veces aparece cada usuario
# Básicamente vamos a hacer el "The word-count problem" con nuestro identificadores.

def countUsers(lista): # Definimos la función que cálcula la profundidad.
    """
    method para contar veces que aparece un usuario
    inputs:
        *lista: lista de usuarios
    output: diccionario de {'ID usuario':profundidad en el proceso}
    """
    hist = {} # Creamos un diccionario vacío
    for user in lista:
        if user in hist:
            hist[user] += 1
        else:
            hist[user] = 1
    return hist

dict_user = countUsers(datos_proceso['identificador']) # aplicamos la función a nuestra lista de usuarios
profundidad = pd.DataFrame(dict_user.items(), columns=['identificador','profundidad']) # Diccionario a dataframe

# Añadimos la nueva variable anuestro dataframe
df = pd.merge(df,profundidad, on ='identificador', how='left')

In [76]:
df.head()

,identificador,ssoo,profundidad
0,100016u,0,1
1,100018u,0,4
2,100026u,1,1
3,100027u,1,1
4,100036u,1,1


El siguiente paso es añadir 12 variables usando el archivo que hemos importado previamente (datos_proceso.csv)
Estas variables representaran el tiempo, número de visitas y número de visitas únicas que cada usuario está en cada página.

Para conseguir esto, tendremos que hacer un merge para cada página del proceso.

In [77]:
# añadimos datos pagina 1
# A la vez que añadimos filtramos por page 1, eliminamos la columna página y
# cambiamos el nombre a las columnas
df = (pd.merge(df, datos_proceso[datos_proceso['página']=='page1'].
               drop('página',1).
                   rename(columns = {'Promedio de tiempo en la página':'avg_tiempo_pag1',
                                     'Número de visitas a páginas':'vistas_pag1',
                                     'Número de visitas a páginas únicas':'vistas_unicas_pag1'}),
               how='left', on='identificador'
              ))


# añadimos datos pagina 2
# A la vez que añadimos filtramos por page 2, eliminamos la columna página y
# cambiamos el nombre a las columnas
df = (pd.merge(df, datos_proceso[datos_proceso['página']=='page2'].
               drop('página',1).
                   rename(columns = {'Promedio de tiempo en la página':'avg_tiempo_pag2',
                                     'Número de visitas a páginas':'vistas_pag2',
                                     'Número de visitas a páginas únicas':'vistas_unicas_pag2'}),
               how='left', on='identificador'
              ))


# añadimos datos pagina 3
# A la vez que añadimos filtramos por page 3, eliminamos la columna página y
# cambiamos el nombre a las columnas
df = (pd.merge(df, datos_proceso[datos_proceso['página']=='page3'].
               drop('página',1).
                   rename(columns = {'Promedio de tiempo en la página':'avg_tiempo_pag3',
                                     'Número de visitas a páginas':'vistas_pag3',
                                     'Número de visitas a páginas únicas':'vistas_unicas_pag3'}),
               how='left', on='identificador'
              ))


# añadimos datos pagina 4
# A la vez que añadimos filtramos por page 4, eliminamos la columna página y
# cambiamos el nombre a las columnas
df = (pd.merge(df, datos_proceso[datos_proceso['página']=='page4'].
               drop('página',1).
                   rename(columns = {'Promedio de tiempo en la página':'avg_tiempo_pag4',
                                     'Número de visitas a páginas':'vistas_pag4',
                                     'Número de visitas a páginas únicas':'vistas_unicas_pag4'}),
               how='left', on='identificador'
              ))

In [78]:
df.head()

,identificador,ssoo,profundidad,avg_tiempo_pag1,vistas_pag1,vistas_unicas_pag1,avg_tiempo_pag2,vistas_pag2,vistas_unicas_pag2,avg_tiempo_pag3,vistas_pag3,vistas_unicas_pag3,avg_tiempo_pag4,vistas_pag4,vistas_unicas_pag4
0,100016u,0,1,15.000000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100018u,0,4,24.000000,1,1,5.0,1.0,1.0,NaN,NaN,NaN,4.0,1.0,1.0
2,100026u,1,1,33.666667,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100027u,1,1,19.000000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100036u,1,1,5.000000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Eliminar NaN de la página 1, explicando el porque y demostrandolo
# Toca hacer una reflexion explicando porque debemos eliminar columnas de la página 4

